# Logistic Regression

In [1]:
import scipy.io as sio
from sklearn.preprocessing import normalize
from sklearn import linear_model as lm
import pandas as pd
import csv
import numpy as np

In [38]:
data_path = 'hashtag_vec.mat'

data = sio.loadmat(data_path)
org_account_type = data['training_labels'].T
org_account_data = data['training_data'] 
indiv_account_data = data['individual_data'] 
indiv_account_order = data['individual_account_order'] 

In [39]:
data.keys()

dict_keys(['__header__', 'individual_data', '__globals__', '__version__', 'training_data', 'training_labels', 'individual_account_order'])

In [40]:
org_account_type.shape

(80, 1)

In [41]:
indiv_account_data.shape

(200, 186)

----
# Build Classifier

In [42]:
# Normalize the data set
normed_org_acnt_data = normalize(org_account_data, axis=1, norm='l2')
normed_indiv_data = normalize(indiv_account_data, axis=1, norm='l2')

In [43]:
# Fit the model to logistic function
logistic = lm.LogisticRegression(fit_intercept=True)
logistic.fit(normed_org_acnt_data, org_account_type)
print('LogisticRegression score: %f'
      % logistic.fit(normed_org_acnt_data, org_account_type).score(normed_org_acnt_data, org_account_type))

LogisticRegression score: 0.975000


/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
logistic.coef_[0][92]

-0.76756998906111373

In [26]:
np.argmax(logistic.coef_)

1

Once we fit the logistic model to the hand labeled popular accounts, we can use the logistic model to predict the type of the individual accounts and the strength of the belief. $logistic.predict$ produces the probability of the sample for each class in the model, where classes are ordered as they are in logistic.classes_ = array([0, 1]), 1 is prolife, 0 is prochoice

In [27]:
# Calculate the strength of the belief for individual accounts
strengths = logistic.predict_proba(normed_indiv_data)

In [28]:
pred = logistic.predict(normed_indiv_data)

In [29]:
file_dict = {}
file_dict['individual_account_order'] = indiv_account_order
file_dict['prochoice_strength'] = strengths[:,0]
file_dict['prolife_strength'] = strengths[:,1]
file_dict['predict_class'] = pred
sio.savemat('indiv_logistic_prop_word.mat', file_dict, do_compression=True)

In [30]:
logistic.classes_

array([0, 1])

# Combine result from following diversity

In [31]:
classifier_df = pd.DataFrame(
    {
    "account_name": [s.strip() for s in indiv_account_order], 
    "prochoice_strength": strengths[:,0],
    "prolife_strength": strengths[:,1],
    "predict_class": pred
    }
)

classifier_df.head()

,account_name,predict_class,prochoice_strength,prolife_strength
0,ABabysRights,1,0.119568,0.880432
1,ABGYNservices,0,0.768067,0.231933
2,Abigailschumach,0,0.634507,0.365493
3,Alboteaparty,0,0.654104,0.345896
4,alexdelong_,0,0.624424,0.375576


There are 197 accounts in the result_df.

In [32]:
classifier_df.shape

(200, 4)

Add a column tells whether the user is a follower of the prolife account (1) or prochoice (-1) account or both (0).

In [33]:
prolife_followers = []
with open('./prolife_followers.csv', 'r') as f:
    reader = csv.reader(f)
    for item in list(reader):
        prolife_followers += item
assert len(set(prolife_followers))==100

prochoice_followers = []
with open('./prochoice_followers.csv', 'r') as f:
    reader = csv.reader(f)
    for item in list(reader):
        prochoice_followers += item
assert len(set(prochoice_followers))==100

classifier_df['follower of'] = [0 if user in prolife_followers and user in prochoice_followers 
                            else 1 if user in prolife_followers else -1 for user in classifier_df['account_name']]

Combine the classifier's result with the following diversity result.

In [34]:
following_proportion = pd.read_csv('proportion.csv')
combine_df = pd.merge(classifier_df, following_proportion, left_on='account_name', right_on='User', how='left')
result_df = combine_df.drop_duplicates()
result_df = result_df.drop('User', axis=1)
print("size of the data frame", result_df.shape)
result_df.tail()

size of the data frame (200, 7)


,account_name,predict_class,prochoice_strength,prolife_strength,follower of,prolife proportion,prochoice proportion
195,VLabarum,0,0.611260,0.388740,1,0.923077,0.076923
196,wafulkerson1,0,0.523748,0.476252,1,1.000000,0.000000
197,WarrenDavidson,0,0.613075,0.386925,1,0.900000,0.100000
198,WilsonRx,0,0.626816,0.373184,1,1.000000,0.000000
199,wordspurify27,1,0.285447,0.714553,-1,0.000000,1.000000


# Save the file

In [35]:
result_df.to_csv("indiv_strength_diversity.csv", index=False)